# Install Earth Engine API and gee
This code checks if the `geemap` package has been installed, and if not it will install `geemap` and its dependencies including earthengine-api, ipyleaflet, and folium.

All code adapted from https://github.com/giswqs/geemap and https://github.com/giswqs/earthengine-py-notebooks by Qiusheng Wu 


In [1]:
import geemap
# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()


Enter verification code: 4/1AX4XfWg2DRhmbf13uRThHJYPpyGFWxybnwvIiYW8c8lyURJhW_mFmFL2Dtw

Successfully saved authorization token.


# Set bounding box for map and image collection


In [2]:
region = ee.Geometry.BBox(-105.89218, 40.27989, -105.17284, 40.72316)

# Get image collection

In [33]:
# first get today's date
from datetime import date

today = date.today()
d1 = today.strftime("%Y-%m-%d")

collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterDate('2021-12-30', '2022-01-30') \
  .filterBounds(region) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
  .median()
  #.map(maskS2clouds) \
  
# median = collection.median()

# Set viz parameters for three different layers

In [4]:
# true color
vizTC = {
    'bands' : ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 6000
}

vizSnow = {
    'bands': ['MSK_SNWPRB'],
    'min': 0,
    'max': 1,
    'palette': ['FFFFFF', 'FF0000']
}

vizFire = {
    'bands': ['B12', 'B8', 'B4'],
    'min': 600,
    'max': 2300
}

# Create an interactive map


In [5]:
Map = geemap.Map(add_google_map = False, layer_ctrl = True)
#center around region of interest
Map.centerObject(region, zoom = 9)

Map.addLayer(collection, vizTC, 'True Color')
Map.addLayer(collection, vizSnow, "Snow Probability")
Map.addLayer(collection, vizFire, "Wildfire")
Map



Map(center=[40.501466743410354, -105.53251000000024], controls=(WidgetControl(options=['position', 'transparen…

# Test out Landsat 8/9

In [24]:
land9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
 .filterDate('2020-01-01', d1) \
  .filterBounds(region)


In [15]:
geemap.image_props(land9.first()).getInfo()

{'ALGORITHM_SOURCE_SURFACE_REFLECTANCE': 'LaSRC_1.5.0',
 'ALGORITHM_SOURCE_SURFACE_TEMPERATURE': 'st_1.3.0',
 'CLOUD_COVER': 18.35,
 'CLOUD_COVER_LAND': 8.48,
 'COLLECTION_CATEGORY': 'T1',
 'COLLECTION_NUMBER': 2,
 'DATA_SOURCE_AIR_TEMPERATURE': 'MODIS',
 'DATA_SOURCE_ELEVATION': 'GLS2000',
 'DATA_SOURCE_OZONE': 'MODIS',
 'DATA_SOURCE_PRESSURE': 'Calculated',
 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT',
 'DATA_SOURCE_WATER_VAPOR': 'MODIS',
 'DATE_ACQUIRED': '2022-04-20',
 'DATE_PRODUCT_GENERATED': 1650605230000,
 'DATUM': 'WGS84',
 'EARTH_SUN_DISTANCE': 1.0046132,
 'ELLIPSOID': 'WGS84',
 'GEOMETRIC_RMSE_MODEL': 9.133,
 'GEOMETRIC_RMSE_MODEL_X': 5.914,
 'GEOMETRIC_RMSE_MODEL_Y': 6.959,
 'GEOMETRIC_RMSE_VERIFY': 6.23,
 'GRID_CELL_SIZE_REFLECTIVE': 30,
 'GRID_CELL_SIZE_THERMAL': 30,
 'GROUND_CONTROL_POINTS_MODEL': 209,
 'GROUND_CONTROL_POINTS_VERIFY': 69,
 'GROUND_CONTROL_POINTS_VERSION': 5,
 'IMAGE_DATE': '2022-04-20',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'L1_DATE_PRODUCT_GE

In [25]:
land9.size().getInfo()

19

Write function to sort collection based on time (miliseconds) each image is from user specified date


In [27]:
def date_fun(image):
    return image.set(
    'dateDist',
    ee.Number(image.get('system:time_start')).subtract(dateOfInterest.millis()).abs()
    )
    

In [66]:
import time

def current_milli_time():
    return round(time.time() * 1000)

dateOfInterest = today.strptime(today,"%Y-%m-%d %H:%M:%S")
ee.Number(land9.first().get('system:time_start')).subtract(dateOfInterest.current_milli_time()).abs()

AttributeError: 'datetime.date' object has no attribute 'strptime'

In [52]:
land9.first().get('system:time_start').getInfo()

1636738689635

In [71]:
dateOfInterest

'2022-04-27 00:00:00'

In [70]:
dateOfInterest = today.strftime("%Y-%m-%d %H:%M:%S")

In [61]:
from datetime import datetime

dt_obj = datetime.strptime('20.12.2016 09:38:42,76',
                           '%d.%m.%Y %H:%M:%S,%f')
millisec = dt_obj.timestamp() * 1000

print(millisec)

1482251922760.0


In [112]:
dImage = ee.Date(land9.first().get('system:time_start'))
dInterest = ee.Date("2021-10-01")

type(dImage)
type(dInterest)


TypeError: an integer is required (got type Date)

In [102]:
(dInterest - dImage)


TypeError: unsupported operand type(s) for -: 'String' and 'String'

AttributeError: 'ImageCollection' object has no attribute 'last'